In [1]:
import numpy as np

from my_stuff import MufexKeys
from quantfreedom.enums import CandleProcessingType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.strategies.strategy import Strategy
from quantfreedom.base import backtest_df_only
from quantfreedom.enums import *
from quantfreedom.helper_funcs import create_os_cart_product_nb

mufex = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
)
symbol = "BTCUSDT"
%load_ext autoreload
%autoreload 2

In [2]:
candles = mufex.get_candles_df(symbol=symbol, timeframe='5m', candles_to_dl=600)

It took 00 mins and 01 seconds to download the candles


In [11]:
candles.iloc[4:204]

,open,high,low,close
timestamp,,,,
2023-10-04 12:55:00,27573.2,27573.2,27543.0,27553.0
2023-10-04 13:00:00,27553.0,27561.5,27518.7,27518.7
2023-10-04 13:05:00,27518.7,27558.4,27516.4,27534.8
2023-10-04 13:10:00,27534.8,27549.8,27534.2,27546.4
2023-10-04 13:15:00,27546.4,27553.4,27510.3,27510.3
...,...,...,...,...
2023-10-05 05:10:00,27663.8,27687.9,27663.0,27680.7
2023-10-05 05:15:00,27680.7,27680.9,27656.1,27656.1
2023-10-05 05:20:00,27656.1,27660.1,27651.4,27657.2


In [3]:
mufex.set_exchange_settings(
    symbol=symbol,
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01, 0.02]) / 100,
    sl_based_on_lookback=np.array([50]),
    risk_reward=np.array([2.0, 5.0]),
    leverage_type=np.array([LeverageStrategyType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossStrategyType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitStrategyType.RiskReward]),
    max_equity_risk_pct=np.array([3]) / 100,
    long_or_short=np.array([LongOrShortType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]) / 100,
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
    static_leverage=np.array([0.0]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
    num_candles=np.array([50,100,200]),
)
os_cart_arrays = create_os_cart_product_nb(
    order_settings_arrays=order_settings_arrays,
)

In [4]:
strategy = Strategy(
    candle_processing_mode=CandleProcessingType.BacktestCandleByCandle,
    candles=candles,
)
backtest_df_only(
    equity=1000.0,
    backtest_settings=backtest_settings,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=mufex.exchange_settings,
    candles=candles,
    strategy=strategy,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 12
Total combinations of settings to test: 72

Total candles: 600
Total candles to test: 43,200


,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
0,3,0,4.0,0.0,25.00,-0.000638,-4.3550,995.644977
1,0,1,7.0,0.0,28.57,-0.001261,1.1573,1001.157308
2,4,1,5.0,0.0,20.00,-0.002230,-15.7870,984.213007
3,5,1,5.0,0.0,20.00,-0.002230,-15.7870,984.213007
4,4,0,5.0,0.0,20.00,-0.006403,-14.3550,985.644977
...,...,...,...,...,...,...,...,...
67,3,2,8.0,0.0,12.50,-0.590668,-81.3561,918.643792
68,4,10,4.0,0.0,25.00,-0.596650,40.2579,1040.257940
69,5,10,4.0,0.0,25.00,-0.596650,40.2579,1040.257940
70,3,4,4.0,0.0,50.00,-0.605843,15.0809,1015.080870
